# Brewery Sales Analysis

Analyzing brewery sales data

## Data Import and Preprocessing

In [1]:
# imports
import numpy as np
import pandas as pd
import calendar
from sqlalchemy import create_engine

In [2]:
# read dataset csv
df = pd.read_csv('brew_analysis.csv')

df.head()
# df.info()
# df.describe()

,SALES_ID,SALES_REP,EMAILS,BRANDS,PLANT_COST,UNIT_PRICE,QUANTITY,COST,PROFIT,COUNTRIES,REGION,MONTHS,YEARS
0,10101,Jardine,jard@gmail.com,trophy,150,200,725,145000,36250,Ghana,Southeast,January,2019
1,10102,Gill,gillhell@uk.com,budweiser,250,500,815,407500,203750,Nigeria,west,February,2018
2,10103,Sorvino,sorvi2000@gmail.com,castle lite,180,450,937,421650,252990,Togo,southsouth,March,2018
3,10104,Jones,jone.ai@yahoo.com,eagle lager,170,250,765,191250,61200,Benin,northwest,April,2018
4,10105,Andrews,andy@gmail.com,hero,150,200,836,167200,41800,Senegal,northeast,May,2017


In [3]:
# fix column name to conventional format (lowercase, singular)
# extract column headers
columns_proper = [x.lower() for x in list(df.columns)]

# remove plural 's'
columns_proper = [x[:-1] if x[-1] == 's' else x for x  in columns_proper]

# replace
columns_proper = [x[:-2]+'y' if x[-2:] == 'ie' else x for x in columns_proper]

print(columns_proper)

['sales_id', 'sales_rep', 'email', 'brand', 'plant_cost', 'unit_price', 'quantity', 'cost', 'profit', 'country', 'region', 'month', 'year']


In [4]:
# update df columns with columns proper
df.columns = columns_proper

df.head()

,sales_id,sales_rep,email,brand,plant_cost,unit_price,quantity,cost,profit,country,region,month,year
0,10101,Jardine,jard@gmail.com,trophy,150,200,725,145000,36250,Ghana,Southeast,January,2019
1,10102,Gill,gillhell@uk.com,budweiser,250,500,815,407500,203750,Nigeria,west,February,2018
2,10103,Sorvino,sorvi2000@gmail.com,castle lite,180,450,937,421650,252990,Togo,southsouth,March,2018
3,10104,Jones,jone.ai@yahoo.com,eagle lager,170,250,765,191250,61200,Benin,northwest,April,2018
4,10105,Andrews,andy@gmail.com,hero,150,200,836,167200,41800,Senegal,northeast,May,2017


In [5]:
# create month number and month abbreviated feature

# month number
df['month_num'] = pd.to_datetime(df['month'], format='%B').dt.month

# month abbreviated
df['month_abbr'] = df['month_num'].apply(lambda x : calendar.month_abbr[x])

df.head()

,sales_id,sales_rep,email,brand,plant_cost,unit_price,quantity,cost,profit,country,region,month,year,month_num,month_abbr
0,10101,Jardine,jard@gmail.com,trophy,150,200,725,145000,36250,Ghana,Southeast,January,2019,1,Jan
1,10102,Gill,gillhell@uk.com,budweiser,250,500,815,407500,203750,Nigeria,west,February,2018,2,Feb
2,10103,Sorvino,sorvi2000@gmail.com,castle lite,180,450,937,421650,252990,Togo,southsouth,March,2018,3,Mar
3,10104,Jones,jone.ai@yahoo.com,eagle lager,170,250,765,191250,61200,Benin,northwest,April,2018,4,Apr
4,10105,Andrews,andy@gmail.com,hero,150,200,836,167200,41800,Senegal,northeast,May,2017,5,May


In [6]:
# Generate new feature tp classify countries as either anglophone or francophone
df['country'].unique()
# array(['Ghana', 'Nigeria', 'Togo', 'Benin', 'Senegal'], dtype=object)

# based on research
anglophone_countries = ["The Gambia", "Sierra Leone", "Liberia", "Ghana", "Nigeria"]
francophone_countries = ["Senegal", "Guinea-Bissau", "Guinea", "Ivory Coast", "Mali", "Togo", "Benin", "Burkina Faso", "Niger", "Chad"]
anglophone_and_francophone_countries = ['Cameroon']

# unique value for language_class column
territories = ['anglophone', 'francophone', 'anglophone_and_francophone']

# create country class query string to be used with np.select
anglophone_query_str = '|'.join(anglophone_countries)
francophone_query_str = '|'.join(francophone_countries)
anglophone_and_francophone_query_str = '|'.join(anglophone_and_francophone_countries)

# classification list quering individual country name in df.country using query string (boolean series)
territory_condition = [
    (df['country'].str.contains(anglophone_query_str)),
    (df['country'].str.contains(francophone_query_str)),
    (df['country'].str.contains(anglophone_and_francophone_query_str))
]

df['territory'] = np.select(territory_condition, territories, default='other')


df['territory'].value_counts()

,count
territory,
francophone,627
anglophone,420


In [7]:
# classify drinks as either malt or not

df['is_malt'] = df['brand'].str.lower().str.contains('malt').apply(lambda x : 1 if x else 0)

df['is_malt'].value_counts()
# df['brand'].value_counts()


,count
is_malt,
0,749
1,298


In [8]:
# verify profit column values
(df['profit'] == df['cost'] - (df['plant_cost'] * df['quantity'])).sum() == df.shape[0]

True

### 1. Within the space of the last three years, what was the profit worth of the breweries, inclusive of the anglophone and the francophone territories?

In [9]:
# dataset spans only 3 years
df['year'].value_counts()

total_profit_worth = df['profit'].sum()

print('total profit worth over the past 3 years is', total_profit_worth)

total profit worth over the past 3 years is 105587420


### 2. Compare the total profit between these two territories (anglophone vs francophone) in order for the territory manager, Mr. Stone make a strategic decision that will aid profit maximization in 2020.

In [10]:
# total profit for each territories
total_profit_by_territory = df.groupby('territory')['profit'].agg(total_profit='sum')

print(total_profit_by_territory)

print()

             total_profit
territory                
anglophone       42389260
francophone      63198160



### 3. What country generated the highest profit in 2019?

In [11]:
# country with the highest profit in 2019
total_profit_by_territory_2019 = df[df['year'] == 2019].groupby('country')['profit'].agg(total_profit_2019='sum').sort_values(by='total_profit_2019', ascending=False).reset_index()



print(total_profit_by_territory_2019, '\n')

max_total_profit_by_territory_2019 = total_profit_by_territory_2019.iloc[0]

print(max_total_profit_by_territory_2019['country'], 'generated the highest profit which was', max_total_profit_by_territory_2019['total_profit_2019'])

   country  total_profit_2019
0    Ghana            7144070
1  Senegal            6687560
2     Togo            6109960
3    Benin            5273340
4  Nigeria            4805320 

Ghana generated the highest profit which was 7144070


### 4. What year saw the highest profit?

In [12]:
# max of profit by year
year_with_highest_profit = df.groupby('year')['profit'].agg(total_profit='sum').sort_values(by='total_profit', ascending=False).index[0]

print(year_with_highest_profit, 'saw the highest profit out of the three.')

2017 saw the highest profit out of the three.


### 5. Which month in the three years was the least profit generated?

In [13]:
# month with the least profit.
month_year_lowest_profit = df.groupby(['year', 'month'])['profit'].agg(total_profit='sum').sort_values(ascending=True, by='total_profit').index[0]
df.groupby(['year', 'month'])['profit'].agg(total_profit='sum').sort_values(ascending=True, by='total_profit')

print(month_year_lowest_profit[1], month_year_lowest_profit[0], 'saw the lowest profit.')

February 2019 saw the lowest profit.


### What was the minimum profit in the month of December 2018?

In [14]:
# minimum profit in the month of December 2018?
min_profit_dec_2019 = df[(df['year'] == 2018) & (df['month'] == 'December')][['sales_rep', 'brand', 'profit']].sort_values(by='profit', ascending=True).iloc[0]

sales_rep, brand, profit = min_profit_dec_2019

print(f'The minimum profit for December 2018, which is {profit} from supplying {brand.capitalize()} was made by {sales_rep}.')

The minimum profit for December 2018, which is 38150 from supplying Hero was made by Jones.


### 7. Compare the profit in percentage for each of the months in 2019

In [15]:
total_profit_cost_2018 = df[df['year'] == 2018].groupby('month').agg(month_num=('month_num', 'first'), total_cost=('cost','sum'), total_profit=('profit', 'sum')).sort_values(by='month_num')

total_profit_cost_2018['pct_profit'] = (total_profit_cost_2018['total_profit'] / total_profit_cost_2018['total_cost'] * 100).round(2)

print(total_profit_cost_2018[['pct_profit']])

           pct_profit
month                
January         42.89
February        43.77
March           42.40
April           44.77
May             39.78
June            45.95
July            43.02
August          39.93
September       43.50
October         46.65
November        43.50
December        48.13


### 8. Which particular brand generated the highest profit in Senegal?

In [16]:
# 8. Which particular brand generated the highest profit in Senegal?
max_profit_brand_senegal = df[df['country'] == 'Senegal'].groupby('brand')['profit'].agg(total_profit=('sum')).sort_values(by='total_profit', ascending=False).index[0]
# df[df['country'] == 'Senegal'].groupby('brand')['profit'].sum().sort_values( ascending=False)[0]

print(max_profit_brand_senegal.capitalize(), 'had the highest profit in Senegal')

Castle lite had the highest profit in Senegal


## SECTION B: BRAND ANALYSIS

### 1. Within the last two years, what are the top three brands consumed in the francophone countries?

In [17]:
# last two years
last_two_years = df['year'].value_counts().sort_index(ascending=False).head(2).index.to_list()
print('last two years are:')
for year in last_two_years: print(year)

# top 3 sold/consumed brands in francophone countries in the last two years
top_brands_sold_frnc_18_19 = df[(df['year'].isin(last_two_years)) & (df['territory'] == 'francophone')].groupby('brand')['quantity'] \
    .agg(unit_sold='sum').sort_values(by='unit_sold', ascending=False).head(3).reset_index().set_index(pd.Index(range(1,4)))

print('\n','top three brands sold in francophone countries during this period:', '\n', top_brands_sold_frnc_18_19)

last two years are:
2019
2018

 top three brands sold in francophone countries during this period: 
          brand  unit_sold
1       trophy      52899
2         hero      50846
3  eagle lager      50630


### 2. What are the top two choices of consumer brands in Ghana?

In [18]:
top_brands_ghana = df[df['country'] == 'Ghana'].groupby('brand')['quantity'].agg(unit_sold='sum').sort_values(by='unit_sold', ascending=False).head(2) \
    .reset_index().set_index(pd.Index(range(1,3)))

print('The top two brand choices in Ghana:', '\n')
print(top_brands_ghana)

The top two brand choices in Ghana: 

         brand  unit_sold
1  eagle lager      25829
2  castle lite      25806


### 3. Find out the details of beers consumed in the past three years in the most oil reached country in West Africa.

In [19]:
# information on beer consumption in Nigeria (most oil riched) during the last three years

df_years = list(df['year'].unique())
df_years.sort(reverse=True)
last_3_years = df_years[:3]

df[(df['country'] == 'Nigeria') & (df['year'].isin(last_3_years)) & (df['is_malt'] == 0)].groupby('brand') \
    .agg(total_quantity_ordered=('quantity', 'sum'), total_revenue=('cost', 'sum'), total_profit=('profit', 'sum')) \
    .reset_index().set_index(pd.Index(range(1, (df[df['is_malt'] == 0]['brand'].nunique() + 1))))
# df[(df['country'] == 'Nigeria')]

# last_3_years

,brand,total_quantity_ordered,total_revenue,total_profit
1,budweiser,26153,13076500,6538250
2,castle lite,25681,11556450,6933870
3,eagle lager,25872,6468000,2069760
4,hero,25811,5162200,1290550
5,trophy,25743,5148600,1287150


### 4. What was the favorites malt brands in Anglophone region between 2018 and 2019?

In [20]:
# Favorites malt brands in Anglophone region between 2018 and 2019

# df.query('year in [2018, 2019] and territory == "anglophone" and is_malt == 1')
fav_malt_anglo_18_19 = df[(df['year'].isin([2018, 2019])) & (df['territory'] == 'anglophone') & (df['is_malt'] == 1)].groupby('brand')['quantity'] \
                        .agg(total_consumed='sum').sort_values(by='total_consumed', ascending=False).reset_index().set_index(pd.Index([1,2]))

fav_malt, fav_malt_amount = fav_malt_anglo_18_19.iloc[0].to_list()

print(fav_malt_anglo_18_19)

print('\n', fav_malt, f'is the favourite malt brand in Anglophone countries between 2018 and 2019. It was bought {fav_malt_amount}.')

        brand  total_consumed
1  grand malt           33221
2   beta malt           32118

 grand malt is the favourite malt brand in Anglophone countries between 2018 and 2019. It was bought 33221.


### 5. Which brands sold the highest in 2019 in Nigeria?

In [21]:
# Which brands sold the highest in 2019 in Nigeria?
ngr_brands_2019 = df[(df['country'] == 'Nigeria') & (df['year'] == 2019)].groupby('brand')['quantity'].agg(number_sold='sum') \
                    .sort_values('number_sold', ascending=False).reset_index().set_index(pd.Index(range(1, df['brand'].nunique() + 1)))

brand, amount = ngr_brands_2019.iloc[0].to_list()

print(ngr_brands_2019)
print('\n', f'{brand.capitalize()} sold the highest in 2019 at {amount}.')


         brand  number_sold
1         hero         9622
2  eagle lager         8401
3    beta malt         8389
4   grand malt         8125
5       trophy         5803
6    budweiser         5490
7  castle lite         3388

 Hero sold the highest in 2019 at 9622.


### 6. What was the favorite brands in the South-South region in Nigeria?

In [22]:
# 6. Favorite brands in the South-South region in Nigeria

fav_brands_ss_ngr = df[(df['country'] == 'Nigeria') & (df['region'] == 'southsouth')].groupby('brand')['quantity'] \
                    .agg(quantity_sold='sum').sort_values('quantity_sold', ascending=False).reset_index() \
                    .set_index(pd.Index(range(1, df['brand'].nunique() + 1)))

print(fav_brands_ss_ngr )

fav_brand = fav_brands_ss_ngr.iloc[0].to_list()

print('\n', '{0} was the favorite brand in South-South Nigeria. It sold {1} units.'.format(*fav_brand))

         brand  quantity_sold
1  eagle lager           4551
2       trophy           4468
3         hero           4456
4    budweiser           4328
5  castle lite           4287
6    beta malt           4257
7   grand malt           3914

 eagle lager was the favorite brand in South-South Nigeria. It sold 4551 units.


### 7. What is the beer consumption in Nigeria?

In [23]:
# 7. Beer consumption in Nigeria

num_of_beer_NGR = df[(df['country'] == 'Nigeria') & (df['is_malt'] == 0)]['quantity'].sum()

print('{0} unit of beer were sold in Nigeria.'.format(num_of_beer_NGR))

129260 unit of beer were sold in Nigeria.


### 8. What is the level of consumption of Budweiser in the regions in Nigeria

In [24]:
# 8. What is the level of consumption of Budweiser in the regions in Nigeria

ngr_bud_qty = df[(df['brand'] == 'budweiser') & (df['country'] == 'Nigeria')].groupby('region')['quantity'].agg(bud_qty_consumed='sum') \
                .reset_index().set_index(pd.Index(range(1, df[(df['country'] == 'Nigeria') & (df['brand'] == 'budweiser')]['region'].nunique() + 1)))

print(ngr_bud_qty)




         region  bud_qty_consumed
1     Southeast              4113
2  northcentral              4498
3     northeast              4320
4     northwest              4274
5    southsouth              4328
6          west              4620


### What is level of consumption of Budweiser in the regions in Nigeria in 2019?

In [25]:
# Level of consumption of Budweiser in the regions in Nigeria in 2019

ngr_bud_qty = df[(df['brand'] == 'budweiser') & (df['country'] == 'Nigeria') & (df['year'] == 2019)].groupby('region')['quantity'].agg(bud_qty_consumed='sum') \
                .reset_index().set_index(pd.Index(range(1, df[(df['country'] == 'Nigeria') & (df['brand'] == 'budweiser') & (df['year'] == 2019)]['region'].nunique() + 1)))

print(ngr_bud_qty)


       region  bud_qty_consumed
1   Southeast              1821
2   northeast               990
3   northwest               948
4  southsouth               847
5        west               884


## SECTION C: COUNTRY ANALYSIS

### 1. What country has the highest consumption of beer?

In [26]:
# 1. Country with the highest consumption of beer
higest_beer = df[df['is_malt'] == 0].groupby('country')['quantity'].agg(total_quantity_consumed='sum').sort_values(by='total_quantity_consumed', ascending=False)

print(higest_beer, '\n\n', higest_beer.index[0], 'has the highest consumption of beer.')

         total_quantity_consumed
country                         
Senegal                   129875
Nigeria                   129260
Benin                     127455
Ghana                     127232
Togo                      125548 

 Senegal has the highest consumption of beer.


### 2. Highest sales personnel of Budweiser in Senegal

In [27]:
# 2. Highest sales personnel of Budweiser in Senegal
sen_bud_rep = df[(df['brand'] == 'budweiser') & (df['country'] == 'Senegal')].groupby('sales_rep')['quantity'].agg(bud_qty_consumed='sum').sort_values(by='bud_qty_consumed', ascending=False)

print(sen_bud_rep, '\n\n', sen_bud_rep.index[0], 'is the highest sales personnel of Budweiser in Senegal.')

           bud_qty_consumed
sales_rep                  
Jones                  5917
Sorvino                4246
Andrews                3336
Kivell                 2813
Jardine                2507
Parent                 1842
Smith                  1673
Gill                    966
Howard                  952
Morgan                  942
Thompson                729 

 Jones is the highest sales personnel of Budweiser in Senegal.


### 3. Country with the highest profit in the fourth quarter in 2019

In [28]:
# 3. Country with the highest profit in the fourth quarter in 2019

q4_profit_by_country = df[df['month_num'].isin([10,11,12])].groupby('country')['profit'].agg(q4_profit='sum').sort_values(by='q4_profit', ascending=False)

print(q4_profit_by_country, '\n\n', q4_profit_by_country.index[0], 'has the highest profit in the fourth quarter in 2019.')

         q4_profit
country           
Nigeria    5519070
Togo       5490840
Benin      5283210
Senegal    5052430
Ghana      5029170 

 Nigeria has the highest profit in the fourth quarter in 2019.
